# 기출4

In [2]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [3]:
dat=pd.read_csv('test/tweets2.csv')
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Data columns (total 13 columns):
No                   313 non-null int64
atConference         313 non-null bool
day                  313 non-null object
impressions          313 non-null int64
retweets             313 non-null int64
likes                313 non-null int64
userprofileclicks    313 non-null int64
urlClicks            313 non-null int64
hashtagClicks        313 non-null int64
detailExpands        313 non-null int64
follows              313 non-null int64
mediaViews           313 non-null int64
mediaEngagements     313 non-null int64
dtypes: bool(1), int64(11), object(1)
memory usage: 29.8+ KB


# 전처리

In [4]:
pre1=dat.copy()
pre1.columns

Index(['No', 'atConference', 'day', 'impressions', 'retweets', 'likes',
       'userprofileclicks', 'urlClicks', 'hashtagClicks', 'detailExpands',
       'follows', 'mediaViews', 'mediaEngagements'],
      dtype='object')

In [5]:
pre1['totalSum'] = pre1.iloc[:, 4:].sum(axis=1)

In [6]:
basetable1=pre1.copy()

# Q1

In [7]:
basetable1.atConference.value_counts()

True     239
False     74
Name: atConference, dtype: int64

In [8]:
q1=basetable1[basetable1['retweets'] > 0]

q1_1=q1[q1['atConference'] == True]['likes']
q1_0=q1[q1['atConference'] == False]['likes']

In [9]:
import scipy.stats as st

In [11]:
q1_out=st.ttest_ind(q1_1, q1_0)
q1_out

Ttest_indResult(statistic=-4.702598503830562, pvalue=1.873518653421921e-05)

# Q2

In [16]:
pre2=basetable1.copy()

In [14]:
pre2.columns

Index(['No', 'atConference', 'day', 'impressions', 'retweets', 'likes',
       'userprofileclicks', 'urlClicks', 'hashtagClicks', 'detailExpands',
       'follows', 'mediaViews', 'mediaEngagements', 'totalSum'],
      dtype='object')

In [18]:
# min_var_list=pre2.columns[3:-1]
min_var_list=pre2.columns.drop(['No', 'atConference', 'day','totalSum' ])
min_var_list

Index(['impressions', 'retweets', 'likes', 'userprofileclicks', 'urlClicks',
       'hashtagClicks', 'detailExpands', 'follows', 'mediaViews',
       'mediaEngagements'],
      dtype='object')

In [19]:
from sklearn.preprocessing import MinMaxScaler

In [20]:
minmax=MinMaxScaler().fit(pre2[min_var_list])
pre2[min_var_list] = minmax.transform(pre2[min_var_list])

In [21]:
pre2.head(2)

,No,atConference,day,impressions,retweets,likes,userprofileclicks,urlClicks,hashtagClicks,detailExpands,follows,mediaViews,mediaEngagements,totalSum
0,1,False,2017-10-26,0.001909,0.000000,0.016667,0.000000,0.0,0.0,0.051282,0.0,0.0,0.0,3
1,2,False,2017-10-26,0.074440,0.029412,0.116667,0.133333,0.0,0.0,0.076923,0.0,0.0,0.0,13


In [22]:
import locale
locale.setlocale(locale.LC_ALL,'')

'Korean_Korea.949'

In [23]:
pre2['weekday']=pd.to_datetime(pre2['day']).dt.strftime('%a')

In [25]:
train=pre2[(pre2['No'] % 4) != 0]
test=pre2[(pre2['No'] % 4) == 0]

In [26]:
from sklearn.neighbors import KNeighborsClassifier

In [27]:
knn=KNeighborsClassifier().fit(train[min_var_list], train['atConference'])

In [28]:
pred=knn.predict(test[min_var_list])

In [29]:
from sklearn.metrics import confusion_matrix

In [30]:
confusion_matrix(test['atConference'], pred)  # 

array([[12,  7],
       [ 2, 57]], dtype=int64)

# Q3

In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
train_d=pd.get_dummies(train, columns=['weekday'], drop_first=True)
test_d=pd.get_dummies(test, columns=['weekday'], drop_first=True)

In [33]:
q3_list=train_d.columns.drop(['No', 'atConference', 'day','totalSum' ])

In [34]:
logit=LogisticRegression(fit_intercept=True, C=100000, random_state=123, penalty='l2', solver='newton-cg')
logit.fit(train_d[q3_list], train_d['atConference'])

LogisticRegression(C=100000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
pred=logit.predict_log_proba(test_d[q3_list])
pred_c=logit.predict(test_d[q3_list])
pred_c

array([False,  True, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False, False, False,  True,  True, False,
       False, False, False, False, False, False])

In [41]:
pred[test_d['atConference'] == True, 1].sum()

-2.8886339119282667

In [42]:
pred[pred_c == True, 1].sum()

-2.3495514037432286

In [43]:
logit.coef_  # 회귀계수

array([[ 2.29929548e+01,  2.71402599e+01, -2.80032182e+01,
        -1.06798177e+01, -3.04891531e+01,  7.05700223e+00,
        -1.28085079e+01, -2.15030646e-02, -6.79306963e+01,
         1.33463240e+02, -4.90105784e-01,  4.49731214e+01,
         4.71370500e+01, -3.50430784e-01, -3.69565805e-01,
         4.74915312e+01]])

In [57]:
# 'retweets'의 odds ratio를 구하시오.

# pd.Series(np.exp(logit.coef_), index=q3_list)['retweets']
pd.Series(np.exp(logit.coef_[0]), index=q3_list)['retweets']

612160231251.0381

In [44]:
logit.intercept_  # 상수항

array([-44.175493])

In [55]:
logit.coef_[0][1]

27.14025990096064

In [56]:
logit.coef_[0,1]

27.14025990096064